In [3]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

from src import mri
from src import rim
from src.utils import PyramidLoss

In [5]:
batch_size = 50

mnist = tf.contrib.learn.datasets.load_dataset("mnist")
inputs = tf.placeholder(shape=[batch_size, 784], dtype=tf.float32)
x = tf.reshape(inputs, [-1, 28, 28, 1])
x = tf.cast(x, tf.complex64)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [6]:
f = mri.MRI()
y = f(x)
y

<tf.Tensor 'add:0' shape=(50, 28, 28, 1) dtype=complex64>

# [Recurrent inference machines](https://openreview.net/forum?id=HkSOlP9lg)



In [8]:
rim = rim.RIM(f.dLdx, 196)
x_t = rim(y)

x, x_t = (tf.real(x), tf.real(x_t))

In [9]:
# new idea. can introduce a loss at every step, rather than just at
# the end. need to test versus loss at end.

def summarise_candidates(model):
    for i, x_t in enumerate(rim.candidate_xs):
        tf.summary.image('x_t/{}'.format(i), tf.real(x_t), max_outputs=1)
        
def candidate_loss(model, x):
    return tf.add_n([tf.losses.mean_squared_error(x, x_t)
              for x_t in rim.candidate_xs])
        
summarise_candidates(rim)
loss = candidate_loss(rim, x)
tf.summary.scalar('loss', loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [9]:
# # tf.summary.image('x_t', x_t)
# # tf.summary.image('x', x)


# loss = tf.losses.mean_squared_error(x, x_t)
# # loss_fn = PyramidLoss(32, 3)
# # loss = loss_fn((x, x_t))
# tf.summary.scalar('loss', loss)


In [10]:
opt = tf.train.AdamOptimizer()
train_step = opt.minimize(loss, var_list=rim.variables)

In [11]:
logdir = "/tmp/rim/candidate/2"

In [12]:
!rm -rf logdir

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir, sess.graph)

In [14]:
for i in range(10000):
    idx = np.random.randint(0, len(mnist.train.images), 50)
    sess.run(train_step, feed_dict={inputs: mnist.train.images[idx, ...]})
    
    if i % 10 == 0:
        L, summary = sess.run([loss, merged], feed_dict={inputs: mnist.test.images[:50, ...]})
        writer.add_summary(summary, i)
        print('\rloss: {}'.format(L), end='', flush=True)

loss: 0.5150448083877563

KeyboardInterrupt: 